In [2]:
%pylab inline
from collections import OrderedDict
import os
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [3]:
def read_coverage_tsv(fp, read_lengths=None):
    df = pd.read_table(fp, header=None)

    df.columns =  ['read_length', 'offset', 'coverage']
    coverages = OrderedDict()
    if read_lengths is None:
        read_lengths = sorted(df.read_length.unique())

    for read_length in read_lengths:
        try:
            df_subset = df[df.read_length==read_length].iloc[0]
        except IndexError:
            print ('IndexError: {} | fp: {} | read_length: {}'.format(df[df.read_length==read_length], fp, read_length))
            raise

        coverage = eval(df_subset.coverage)
        coverage = pd.Series(coverage, index=np.arange(-df_subset.offset, len(coverage)-df_subset.offset))
        coverages[read_length]  = coverage
    return coverages

In [4]:
assembly_common_name_map = {'mm10': 'mouse', 'hg38': 'human'}
sheet_name = 'mm10'
root_dir = '/staging/as/wenzhenl/re-ribo-analysis/'
excel_df = pd.read_excel('../data/datasets_info.xlsx', sheet_name=sheet_name)
excel_df.head()



,GEO,SRA,RIBO sample,RNA sample,read lengths,offsets,RIBO adapter,RNA adapter,RIBO strandedness,RNA strandedness
0,GSE22001,SRP003554,SRX026871,SRX026872,"28,29,30","12,12,12",TCGTATG,TCGTATG,forward,forward
1,GSE72064,SRP062407,SRX1149649,SRX1149668,"28,29,30,31","12,12,12,12",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
2,GSE84112,SRP078005,SRX1900396,SRX1900402,"26,27,28,29,30","12,12,12,12,12",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward
3,GSE89011,SRP091889,SRX2255510,SRX2255511,"26,27,28,29,30","12,12,12,12,12",AGATCGGAAGAGCACACGTCT,AGATCGGAAGAGCACACGTCT,forward,forward
4,GSE102890,SRP115915,SRX3110803,SRX3110807,"29,30,31,32,33,34","12,12,12,13,13,13",TGGAATTCTCGGGTGCCAAGG,TGGAATTCTCGGGTGCCAAGG,forward,forward


In [ ]:
for index, row in excel_df.iterrows():
    srp = row['SRA']
    ribo_srx = row['RIBO sample'].strip().split(' ')[0]
    offsets = list(map(lambda x: int(x.strip()), row['offsets'].split(',')))    
    
    read_lengths = list(map(lambda x: int(x.strip()), row['read lengths'].split(',')))
    read_lengths_offsets = dict(zip(read_lengths, offsets))
    
    ribo_tsv_fp = os.path.join(root_dir,
                               '{}_{}'.format(srp, assembly_common_name_map[sheet_name]), 
                               'mapped',
                               'orfs',
                               '{}_metagene_profiles_5p.tsv'.format(ribo_srx))
    
    
    if not os.path.isfile(ribo_tsv_fp):
        print('Missing {}'.format(ribo_tsv_fp))
        continue
    try:
        rna_coverages = read_coverage_tsv(rna_tsv_fp)
    except pd.errors.EmptyDataError:
        print ('EmptyDataError: {}'.format(rna_tsv_fp))
        continue
    
    for ribode
    